In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import shutil
import difflib
from pulp import *

today = date.today()
today = today.strftime("%m.%d.20%y")
today

'12.13.2021'

In [4]:
downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis'
daily_file = "final_PlayerPool_12.13.2021.xlsx"

os.chdir(data_loc)

df = pd.read_excel(daily_file)

In [5]:
df.head()
df['Roster Position'] = df['Roster Position'].str.split('/')
df = df.explode('Roster Position')
df.head()

,Rank,Name_x,Team,Pos,Opp,p,3,r,a,s,...,fta,to,fgm,ftm,Expected_Points,nameTeam,Name_y,Salary,Roster Position,Value
0,1,Nikola Jokic,DEN,PF/C,WAS,29.57,1.63,12.66,8.62,1.09,...,6.02,3.05,11.56,4.82,61.6550,Nikola Jokic_DEN,Nikola Jokic,12000,C,5.137917
0,1,Nikola Jokic,DEN,PF/C,WAS,29.57,1.63,12.66,8.62,1.09,...,6.02,3.05,11.56,4.82,61.6550,Nikola Jokic_DEN,Nikola Jokic,12000,UTIL,5.137917
1,2,Stephen Curry,GSW,PG/SG,@ IND,28.17,5.55,5.73,6.19,1.63,...,3.95,2.62,9.50,3.62,49.9625,Stephen Curry_GSW,Stephen Curry,11400,PG,4.382675
1,2,Stephen Curry,GSW,PG/SG,@ IND,28.17,5.55,5.73,6.19,1.63,...,3.95,2.62,9.50,3.62,49.9625,Stephen Curry_GSW,Stephen Curry,11400,G,4.382675
1,2,Stephen Curry,GSW,PG/SG,@ IND,28.17,5.55,5.73,6.19,1.63,...,3.95,2.62,9.50,3.62,49.9625,Stephen Curry_GSW,Stephen Curry,11400,UTIL,4.382675


In [7]:
# Get a list of players
players = list(df['Name_x'])

# Initialize Dictionaries for Salaries and Positions
salaries = dict(zip(players, df['Salary']))
positions = dict(zip(players, df['Roster Position']))

# Dictionary for Projected Score for each player
project_points = dict(zip(players, df['Expected_Points']))

# Set Players to Take either 1 or 0 values (owned or not)
player_vars = LpVariable.dicts("Player", players, lowBound=0, upBound=1, cat='Integer')

In [8]:
total_score = LpProblem("Fantasy_Points_Problem", LpMaximize)

total_score += lpSum([project_points[i] * player_vars[i] for i in player_vars])

total_score += lpSum([salaries[i] * player_vars[i] for i in player_vars]) <= 50000

In [9]:
# Get indices of players for each position
pg = [p for p in positions.keys() if positions[p] == 'PG']
sg = [p for p in positions.keys() if positions[p] == 'SG']
g =  [p for p in positions.keys() if positions[p] == 'G']
sf = [p for p in positions.keys() if positions[p] == 'SF']
pf = [p for p in positions.keys() if positions[p] == 'PF']
f = [p for p in positions.keys() if positions[p] == 'F']
c =  [p for p in positions.keys() if positions[p] == 'C']
# Set Constraints
total_score += lpSum([player_vars[i] for i in pg]) == 1
total_score += lpSum([player_vars[i] for i in sg]) == 1
total_score += lpSum([player_vars[i] for i in g]) == 1
total_score += lpSum([player_vars[i] for i in sf]) == 1
total_score += lpSum([player_vars[i] for i in f]) == 1
total_score += lpSum([player_vars[i] for i in pf]) == 1
total_score += lpSum([player_vars[i] for i in c]) == 1

In [10]:
total_score.solve()

-1

In [11]:
for v in total_score.variables():
    if v.varValue > 0:
        print(v.name)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

In [10]:
df.dtypes

Rank                 int64
Name                object
Team                object
Pos                 object
Opp                 object
p                  float64
3                  float64
r                  float64
a                  float64
s                  float64
b                  float64
fg%                float64
fga                float64
ft%                float64
fta                float64
to                 float64
fgm                float64
ftm                float64
Expected_Points    float64
Salary               int64
Roster Position     object
Value              float64
dtype: object